> Enterprise Web C#

# Chapter 9 - Data, the new raw material - Dapper

## Introduction

Dapper is an object-relational mapper (ORM) for the .NET ecosystem. It provides a framework for mapping an object-oriented domain model to a traditional relational database.

* It provides a mapping between databases and .NET objects
* It owns the title of King of Micro ORM in terms of speed and is as fast as using a raw ADO.NET data reader
* It extends the `IDbConnection` by providing useful extension methods to query your database.

Dapper is really powerful, but still requires knowledge of SQL in order to be able to execute queries. In a few moments, you'll discover Entity Framework (EF) which removes the need to write any SQL queries at all.

## Installation

Issue the following commands inside a project to install Dapper:

```
dotnet add package Dapper
dotnet add package System.Data.SqlClient
```

Or use these commands inside a .NET Interactive Notebook:

In [ ]:
#r "nuget:Dapper"
#r "nuget:System.Data.SqlClient"

## The database

We are using the [northwind-pubs dataset](https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/northwind-pubs). Follow the step below to import this dataset.

### Windows

1. Download `instnwnd.sql` from the [northwind-pubs dataset](https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/northwind-pubs)
2. Open SQL Server Management Studio and connect to your local instance and right click on `Databases`
3. Open and run the `instnwnd.sql` script
4. Wait patiently for the database to be restored

### macOS/Linux

> This tutorial assumes you have [Docker](https://www.docker.com/get-started) installed.

1. Boot docker and start the SQL Server

```{bash}
docker run -e "ACCEPT_EULA=Y" -e "SA_PASSWORD=Password123\!" -p 1433:1433 -d mcr.microsoft.com/mssql/server:2019-latest
```

2. Download `instnwnd.sql` from the [northwind-pubs dataset](https://github.com/microsoft/sql-server-samples/tree/master/samples/databases/northwind-pubs)
3. Open Azure Data Studio (enable Preview features if he asks for it)
4. Create a new connection to the database:
    - Server: `localhost`
    - Authentication type: `SQL Login`
    - Username: `sa`
    - Password: `Password123!` (ask to remember)
    - Leave the rest on the default
3. Open and run the `instnwnd.sql` script
4. Wait patiently for the database to be restored


## Our first query

In this database a `Categories` table exists. In order to be able to query the table, we need to create a class representing one entity in this table. Dapper will automatically convert the returned data into instances of that class. The `ToString` is only present for pretty output later on. Run the following blocks and look at the magic yourself. **Uncommment the right `connectionString` for your operation system.**

In [ ]:
class Category
{
    public int CategoryID { get; set; }
    public string CategoryName { get; set; }
    public string Description { get; set; }
    public byte[] Picture { get; set; }

    public override string ToString() => $"{CategoryID} - {CategoryName}";
}

In [ ]:
// Windows:
var connectionString = "Server=localhost;Database=Northwind;Trusted_Connection = True;";

// macOS/Linux:
// var connectionString = "Data Source=127.0.0.1,1433;Initial Catalog=Northwind;User ID=sa;Password=Password123!";

In [ ]:
using Dapper;
using System;
using System.Collections.Generic;
using System.Data;
using System.Data.SqlClient;
using System.Linq;

using (IDbConnection db = new SqlConnection(connectionString))
{
    var sql = "SELECT * FROM [dbo].[Categories]";
    List<Category> categories = db.Query<Category>(sql).ToList();
    categories.ForEach(Console.WriteLine);
    // You should see 8 categories in the output
}

This query fetched all 8 categories and automagically mapped the query results to our new `Category` class. Isn't that neat?

This was a synchronous call to the database because we did not use `await`. But there is a `QueryAsync` method just for that, let's try this one out.

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
    var sql = "SELECT * FROM [dbo].[Categories]";
    List<Category> categories = (await db.QueryAsync<Category>(sql)).ToList();
    categories.ForEach(Console.WriteLine);
    // You should again see 8 categories in the output
}

This accomplishes the same result, what would you expect...

For the purpose of these demos the synchronous version is good enough, but we'll use the asynchronous one to get used to async programming.

## Add query parameters

Let's create a query to fetch one category in particular, you need query parameters for this. Query parameters are indicated with an `@`, in this example `@CategoryID` needs to be passed to the query. Therefor we add an anonymous object containing this query parameter as second parameter to the function.

There is also a `QueryFirstOrDefaultAsync` which returns the default value for the given type if nothing matched the query.

The `QueryFirstAsync` won't throw an error if not exactly one result matched. It simply returns the first result if there is any, it throws if there are no results.

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  var sql = "SELECT * FROM [dbo].[Categories] WHERE CategoryID = @CategoryID";
  var category = await db.QueryFirstAsync<Category>(sql, new { CategoryID = 5 });
  Console.WriteLine(category);
}


The next example tries to fetch an unexisting category with `QuerySingleAsync` which will throw an error as this method expects exactly one result. You'll get an error if there is no query result or more than one query result.

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  var sql = "SELECT * FROM [dbo].[Categories] WHERE CategoryID = @CategoryID";
  var category = await db.QuerySingleAsync<Category>(sql, new { CategoryID = 1000 });
  Console.WriteLine(category);
}

## Insert data

Inserting data would require us to write an `INSERT` query with a query parameter for each field in the table. It would look something like this:

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  var sql = "INSERT INTO [dbo].[Categories](CategoryName, Description) VALUES (@CategoryName1, @Description1), (@CategoryName2, @Description2)";
  await db.QueryAsync<Category>(sql, new
  {
    CategoryName1 = "Test 1",
    Description1 = "Description 1",
    CategoryName2 = "Test 2",
    Description2 = "Description 2"
  });
}

There is a much simpler solution for this using the `Dapper.Contrib` extension, let's install it.

In [ ]:
#r "nuget:Dapper.Contrib"

With `Dapper.Contrib` we can create a class of a certain table and tell Dapper how this class is mapped to the database. Take a look at this example. This defines a new class for the `Categories` table and tells Dapper the table name and primary key with some annotations.

In [ ]:
using Dapper.Contrib.Extensions;

[Table("[dbo].[Categories]")]
class Category
{
  [Key]
  public int CategoryID { get; set; }
  public string CategoryName { get; set; }
  public string Description { get; set; }
  public byte[] Picture { get; set; }

  public override string ToString() => $"{CategoryID} - {CategoryName}";
}

Then we can easily add two new categories.

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  
  await db.InsertAsync(new List<Category>
  {
    new Category
    {
      CategoryName = "Fancy category"
    },
    new Category
    {
      CategoryName = "Another one"
    }
  });
}

Check in SQL Server Management Studio or Azure Data Studio that the two new categories were inserted.

## Relations

Dapper has also some basic support for relations. Let's look at this `Product` class. It has a one-to-one relation with `Category`: one product has only one category.

In [ ]:
class Product
{
  public int ProductID { get; set; }
  public string ProductName { get; set; }
  public int SupplierID { get; set; }
  public int CategoryID { get; set; }
  public string QuantityPerUnit { get; set; }
  public decimal UnitPrice { get; set; }
  public int UnitsInStock { get; set; }
  public int UnitsOnOrder { get; set; }
  public int ReorderLevel { get; set; }
  public bool Discontinued { get; set; }
  public Category Category { get; set; }
  public override string ToString() => $"{ProductID} - {ProductName} - {Category.CategoryName}";
}

Let's now fetch each product with its category, ordered by the product name. Therefor we need to pass a mapper function to the `QueryAsync` function which groups by the `CategoryID` and gives us a product and a category as an argument. Also notice that the types for `QueryAsync` have been extended to three types instead of one.

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  var sql = "SELECT * FROM [dbo].[Products] p JOIN [dbo].[Categories] c ON c.CategoryID = p.CategoryID ORDER BY ProductName";
  var products = await db.QueryAsync<Product, Category, Product>(
    sql,
    (product, category) =>
    {
        product.Category = category;
        return product;
    },
    splitOn: "CategoryID"
  );
  Console.WriteLine("\nProducts & categories");
  Console.WriteLine("---------------------");
  foreach (var p in products)
  {
      Console.WriteLine(p);
  }
}

You see... The fun disappears the longer you use Dapper. So be sure it's the right choice for your project. Big and complex project don't benefit from using Dapper, use Entity Framework instead. Maybe with the Dapper extension for Entity Framework.